- PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
- HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
- CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
- Destination - The planet the passenger will be debarking to.
- Age - The age of the passenger.
- VIP - Whether the passenger has paid for special VIP service during the voyage.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- Name - The first and last names of the passenger.
- Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

Plan:
1) Cabin split into deck & num  & side
2) delete `Name`
3) PassengerId split into group & pp
4) CryoSleep, VIP, HomePlanet, Destination and parts of passenger and cabin to labels
5) NaNs

In [143]:
# import libraries
import numpy as np
import pandas as pd
import datetime

from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score

RANDOM_STATE = 654321

In [144]:
train = pd.read_csv('data/train.csv', index_col=None)
test = pd.read_csv('data/test.csv', index_col=None)
df = pd.concat([train, test]).reset_index(drop=True)
train_index = ~df.Transported.isna()
test_index = df.Transported.isna()
print(df.info())
# print(test_index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
 13  Transported   8693 non-null   object 
dtypes: float64(6), object(8)
memory usage: 1.4+ MB
None


In [145]:
def cabin2dns(row):
    # print(row['Cabin'])
    if row['Cabin'] is not np.NAN:
        a = row['Cabin'].split('/')
        row['Deck'] = a[0]
        row['Num'] = int(a[1])
        row['Side'] = a[2]
    else:
        row['Deck'] = np.NAN
        row['Num'] = np.NAN
        row['Side'] = np.NAN
    return row

In [146]:
def passenger2grouppp(row):
    if row['PassengerId'] is not np.NAN:
        a = row['PassengerId'].split('_')
        row['Group'] = int(a[0])
        row['Pp'] = int(a[1])
    else:
        row['Group'] = np.NAN
        row['Pp'] = np.NAN
    return row

In [147]:
df = df.apply(cabin2dns, axis=1)
df.drop(columns=['Cabin'], inplace=True)

In [148]:
df = df.apply(passenger2grouppp, axis=1)
# df.drop(columns=['PassengerId'], inplace=True)

In [149]:
df.drop(columns=['Name'], inplace=True)

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Destination   12696 non-null  object 
 4   Age           12700 non-null  float64
 5   VIP           12674 non-null  object 
 6   RoomService   12707 non-null  float64
 7   FoodCourt     12681 non-null  float64
 8   ShoppingMall  12664 non-null  float64
 9   Spa           12686 non-null  float64
 10  VRDeck        12702 non-null  float64
 11  Transported   8693 non-null   object 
 12  Deck          12671 non-null  object 
 13  Num           12671 non-null  float64
 14  Side          12671 non-null  object 
 15  Group         12970 non-null  int64  
 16  Pp            12970 non-null  int64  
dtypes: float64(7), int64(2), object(8)
memory usage: 1.7+ MB


In [151]:
group_np = df[['Group', 'Pp']].groupby('Group', as_index=False).count()

# group_np

In [152]:
def count_group(row):
    global group_np
    row['Group_count'] = group_np[group_np.Group == row.Group].reset_index().Pp[0]
    return row

In [153]:
df = df.apply(count_group, axis=1)
df

/home/nikolay/.local/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/home/nikolay/.local/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Pp,Group_count
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0.0,P,1,1,1
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0.0,S,2,1,1
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0.0,S,3,1,2
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0.0,S,3,2,2
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1.0,S,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,9266_02,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,NaN,G,1496.0,S,9266,2,2
12966,9269_01,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,NaN,NaN,NaN,NaN,9269,1,1
12967,9271_01,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,NaN,D,296.0,P,9271,1,1
12968,9273_01,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,NaN,D,297.0,P,9273,1,1


In [154]:
categorial = ['CryoSleep', 'VIP', 'HomePlanet', 'Destination', 'Deck', 'Side']

In [155]:
ordinal = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=10000)
ordinal.fit(df[categorial])
df.loc[:,categorial] = ordinal.transform(df.loc[:,categorial])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  float64
 2   CryoSleep     12660 non-null  float64
 3   Destination   12696 non-null  float64
 4   Age           12700 non-null  float64
 5   VIP           12674 non-null  float64
 6   RoomService   12707 non-null  float64
 7   FoodCourt     12681 non-null  float64
 8   ShoppingMall  12664 non-null  float64
 9   Spa           12686 non-null  float64
 10  VRDeck        12702 non-null  float64
 11  Transported   8693 non-null   object 
 12  Deck          12671 non-null  float64
 13  Num           12671 non-null  float64
 14  Side          12671 non-null  float64
 15  Group         12970 non-null  int64  
 16  Pp            12970 non-null  int64  
 17  Group_count   12970 non-null  int64  
dtypes: float64(13), int64(3), 

/tmp/ipykernel_7685/3835204275.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,categorial] = ordinal.transform(df.loc[:,categorial])


In [156]:
label = LabelEncoder()
label.fit(df.Transported)
df.loc[:,'Transported'] = label.transform(df.loc[:,'Transported'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  float64
 2   CryoSleep     12660 non-null  float64
 3   Destination   12696 non-null  float64
 4   Age           12700 non-null  float64
 5   VIP           12674 non-null  float64
 6   RoomService   12707 non-null  float64
 7   FoodCourt     12681 non-null  float64
 8   ShoppingMall  12664 non-null  float64
 9   Spa           12686 non-null  float64
 10  VRDeck        12702 non-null  float64
 11  Transported   12970 non-null  int64  
 12  Deck          12671 non-null  float64
 13  Num           12671 non-null  float64
 14  Side          12671 non-null  float64
 15  Group         12970 non-null  int64  
 16  Pp            12970 non-null  int64  
 17  Group_count   12970 non-null  int64  
dtypes: float64(13), int64(4), 

/tmp/ipykernel_7685/1975810753.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'Transported'] = label.transform(df.loc[:,'Transported'])


In [157]:
df['Age'] = df.Age.median()

In [158]:
def fill_cryo(row):
    costs = row[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum()
    if costs > 0:
        row['CryoSleep'] = 1
    else:
        row['CryoSleep'] = 0
    return row

In [159]:
df = df.apply(fill_cryo, axis=1)

In [160]:
ds = df.drop(columns=['Transported', 'PassengerId'])
ds.dropna(inplace=True)

In [161]:
def predict_categorial(field: str):
    global df
    global ds
    ds_y = ds[field]
    ds_x = ds.drop(columns=[field])
    ds_model = LGBMClassifier(random_state=RANDOM_STATE)
    ds_model.fit(ds_x, ds_y)
    ds_pred = ds_model.predict(ds_x)
    print(accuracy_score(ds_y, ds_pred))
    df.loc[df[field].isna(), field] = ds_model.predict(df.loc[df[field].isna(), :].drop(columns=[field, 'PassengerId', 'Transported']))

In [162]:
predict_categorial('VIP')

0.9966018501038324


In [163]:
df.isna().sum()

PassengerId       0
HomePlanet      288
CryoSleep         0
Destination     274
Age               0
VIP               0
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Transported       0
Deck            299
Num             299
Side            299
Group             0
Pp                0
Group_count       0
dtype: int64

In [164]:
train = df[train_index]
test = df[test_index]

In [165]:
train.dropna(inplace=True)

/tmp/ipykernel_7685/167719433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.dropna(inplace=True)


In [166]:
y_train = train.Transported
X_train = train.drop(columns=['Transported'])
X_test = test.drop(columns=['Transported'])

In [167]:
model = LGBMClassifier(random_state=RANDOM_STATE)

In [168]:
model.fit(X_train.drop(columns=['PassengerId']), y_train)

LGBMClassifier(random_state=654321)

In [169]:
feature_imp = []
for i, f in enumerate(X_train.drop(columns=['PassengerId']).columns):
    feature_imp += [(model.feature_importances_[i], f)]
feature_imp = pd.DataFrame(feature_imp, columns=['importance', 'feature'])
feature_imp = feature_imp.sort_values(by='importance', ascending=False)
feature_imp

,importance,feature
11,455,Num
13,396,Group
9,372,VRDeck
8,345,Spa
6,284,FoodCourt
5,271,RoomService
7,236,ShoppingMall
10,187,Deck
12,124,Side
2,92,Destination


In [170]:
pred_train = model.predict(X_train.drop(columns=['PassengerId']))

In [171]:
accuracy_score(y_train, pred_train)

0.8891334250343879

In [172]:
X_test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep         0
Destination      92
Age               0
VIP               0
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Deck            100
Num             100
Side            100
Group             0
Pp                0
Group_count       0
dtype: int64

In [173]:
pred_test = model.predict(X_test.drop(columns=['PassengerId']))

In [174]:
pred = pd.concat([X_test['PassengerId'].reset_index(drop=True), pd.DataFrame(pred_test)], axis=1)
pred.columns = ['PassengerId', 'Transported']
pred

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1
...,...,...
4272,9266_02,1
4273,9269_01,0
4274,9271_01,1
4275,9273_01,1


In [175]:
def class2bool(row):
    row['Transported'] = 'True' if row['Transported'] == 1 else 'False'
    return row

In [176]:
pred = pred.apply(class2bool, axis=1)
pred

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [177]:
pred.to_csv(f'data/attempt{str(datetime.datetime.now())}.csv', index=False, sep=',', header=True)